# 2020.01.03

# sql 작동시키기 

In [9]:
import pymysql
conn = pymysql.connect(host='192.168.1.243',
                      user='root', port=3306, password='big2019',db='acornschool',
                       charset='utf8mb4', use_unicode=True,
                       cursorclass=pymysql.cursors.DictCursor)

cur = conn.cursor()    # cursor를 해줘야 내가 원하는 데이터를 가져다 가리키는 역할. 

# 테이블 만들기
cur.execute("""CREATE TABLE pet(
           id INT NOT NULL AUTO_INCREMENT,
           name VARCHAR(20),
           owner VARCHAR(20),
           species VARCHAR(20),
           sex CHAR(1),
           birth DATE, 
           death DATE,
           PRIMARY KEY(id)) ENGINE=InnoDB AUTO_INCREMENT=1""")

print(cur.description)
conn.close     # 꼭 close 해줘야한다. 

None


<bound method Connection.close of <pymysql.connections.Connection object at 0x000001F92C871F88>>

# 데이터 입력

In [12]:
conn = pymysql.connect(host='192.168.1.243',
                      user='root', port=3306, password='big2019',db='acornschool',
                       charset='utf8mb4', use_unicode=True,
                       cursorclass=pymysql.cursors.DictCursor)

cur = conn.cursor()

sql = """INSERT INTO pet (name, owner, species, sex, birth, death) VALUES (%s, %s,%s,%s,%s,%s)"""
cur.execute(sql, ('유리','acorn','고양이','f','2014-1-1','2016-2-1'))
cur.execute(sql, ('귀염','acorn','강아지','f','2014-3-1','2016-9-1'))
cur.execute(sql, ('돌이','acorn','곰','f','2014-8-1','2016-2-1'))
conn.commit()
cur.executemany(     # DB :NULL -> Python:None
                """INSERT INTO pet( name,owner, species, sex, birth, death) VALUES (%s, %s,%s,%s,%s,%s)""",
[('대한','민주','cat','f','2001-02-04', None),
('민국','자유','cat','f','2010-03-17', None),
('만세','민주','cat','f','2010-05-13', None),
('하자','민주','cat','m','2015-08-27', None),
('성공','자유','cat','m','2017-08-31', '2018-04-29')])
conn.commit()
print(cur.rowcount)  # 에러가 났는지 데이터가 들어갔는지 확인하기 -> rowcount
conn.close

5


<bound method Connection.close of <pymysql.connections.Connection object at 0x000001F92C863548>>

# 데이터 가져오기

In [15]:
conn = pymysql.connect(host='192.168.1.243',
                      user='root', port=3306, password='big2019',db='acornschool',
                       charset='utf8mb4', use_unicode=True,
                       cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()

cur.execute('select * from pet')   # RecordSet 생성
print(cur.rowcount)  # 8
cur.description  # 설명

cur.fetchone()  # 첫줄
cur.fetchone()  # 두번째 줄
cur.fetchone()  # 세번째 줄
recs = cur.fetchmany(2)  # 다음 2개 
for rec in recs:
    print(rec)
recs = cur.fetchall()    # 나머지 다 가져오기
for rec in recs:
    print(rec)
    
cur.close()
conn.close()

8
{'id': 9, 'name': '대한', 'owner': '민주', 'species': 'cat', 'sex': 'f', 'birth': datetime.date(2001, 2, 4), 'death': None}
{'id': 10, 'name': '민국', 'owner': '자유', 'species': 'cat', 'sex': 'f', 'birth': datetime.date(2010, 3, 17), 'death': None}
{'id': 11, 'name': '만세', 'owner': '민주', 'species': 'cat', 'sex': 'f', 'birth': datetime.date(2010, 5, 13), 'death': None}
{'id': 12, 'name': '하자', 'owner': '민주', 'species': 'cat', 'sex': 'm', 'birth': datetime.date(2015, 8, 27), 'death': None}
{'id': 13, 'name': '성공', 'owner': '자유', 'species': 'cat', 'sex': 'm', 'birth': datetime.date(2017, 8, 31), 'death': datetime.date(2018, 4, 29)}


In [22]:
# 문제 : 생일이 

conn = pymysql.connect(host='192.168.1.243',
                      user='root', port=3306, password='big2019',db='acornschool',
                       charset='utf8mb4', use_unicode=True,
                       cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()


cur.execute("""SELECT *FROM pet WHERE birth > '2015-01-01';""")
row = cur.fetchall()
for pet in row:
    pet_val = list(pet.values())
    print("ID=%d, Name=%s, 주인=%s" % (pet_val[0],pet_val[1],pet_val[2]))


ID=12, Name=하자, 주인=민주
ID=13, Name=성공, 주인=자유
0


In [26]:
# 문제 : 유리를 초심로 update 하시오    
sql = 'update pet set name=%s where name = %s'
cur.execute(sql, ('초심','유리'))
conn.commit()
print(cur.rowcount)

# 항상 커서 오픈을 해야 작동한다. 그리고 끝낼때 클로즈를 해줘야한다. (위에, 밑에 스크립트에 열고, 닫아 줌)

ProgrammingError: Cursor closed

In [25]:
# 문제 : 유리를 삭제하시오 ( 쿼리를 일반화 )
sql='delete from pet where name=%s'
cur.execute(sql,'초심')    # execute : 커리한테 명령을 줄때 사용
conn.commit()
print(cur.rowcount)

cur.close()
conn.close()

0


In [27]:
conn = pymysql.connect(host='192.168.1.243',
                      user='root', port=3306, password='big2019',db='acornschool',
                       charset='utf8mb4', use_unicode=True,
                       cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()
# 여기까지가 모두가 커서 열기


cur.callproc('pet_select')
if(cur.rowcount):
    print(cur.fetchall())
    
# 밑에가 클로즈    
cur.close()
conn.close()

# 이거는 데이터를 불러오는거니까 commit 이 필요없음

[{'id': 7, 'name': '귀염', 'owner': 'acorn', 'species': '강아지', 'sex': 'f', 'birth': datetime.date(2014, 3, 1), 'death': datetime.date(2016, 9, 1)}, {'id': 8, 'name': '돌이', 'owner': 'acorn', 'species': '곰', 'sex': 'f', 'birth': datetime.date(2014, 8, 1), 'death': datetime.date(2016, 2, 1)}, {'id': 9, 'name': '대한', 'owner': '민주', 'species': 'cat', 'sex': 'f', 'birth': datetime.date(2001, 2, 4), 'death': None}, {'id': 10, 'name': '민국', 'owner': '자유', 'species': 'cat', 'sex': 'f', 'birth': datetime.date(2010, 3, 17), 'death': None}, {'id': 11, 'name': '만세', 'owner': '민주', 'species': 'cat', 'sex': 'f', 'birth': datetime.date(2010, 5, 13), 'death': None}, {'id': 12, 'name': '하자', 'owner': '민주', 'species': 'cat', 'sex': 'm', 'birth': datetime.date(2015, 8, 27), 'death': None}, {'id': 13, 'name': '성공', 'owner': '자유', 'species': 'cat', 'sex': 'm', 'birth': datetime.date(2017, 8, 31), 'death': datetime.date(2018, 4, 29)}]


# insert

In [33]:
# 문제 : insert를 하는 프로시져를 생성한 다음
# ('마음이','acorn','백구','f','2018-01-02','2018-11-04') 데이터를 입력하시오
conn = pymysql.connect(host='192.168.1.243',
                      user='root', port=3306, password='big2019',db='acornschool',
                       charset='utf8mb4', use_unicode=True,
                       cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()


data = ('마음이','acorn','백구','f','2018-01-02','2018-11-04')
cur.callproc('pet_insert', data)
conn.commit()    # commit 을 해야 데이터가 새로 추가된다.  // commit 을해야 함수가 실행 된다. 

cur.close()
conn.close()


In [44]:
# 문제 : 이름을 update 하는 프로시져를 작성하고 마음이를 마음으로 수정하시오.




#cur.callproc('pet_update')
#conn.commit()   # commit 을해야 함수가 실행 된다.

cur.close()
conn.close()

# update

In [46]:
conn = pymysql.connect(host='192.168.1.243',
                      user='root', port=3306, password='big2019',db='acornschool',
                       charset='utf8mb4', use_unicode=True,
                       cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()


data = ('마음이','마음', 0)    # _1,_2,_3     ??
cur.callproc('pet_update', data)
cur.execute('SELECT @_pet_update_2')   # _2번째로 바꿔라
conn.commit()
result = cur.fetchone()
print(result['@_pet_update_2'])
if result['@_pet_update_2'] == 2:
    print('수정할 이름이 없습니다')
else:
    print('성공적으로 수정되었습니다')

cur.close()
conn.close()

0
성공적으로 수정되었습니다


# delete

In [50]:
conn = pymysql.connect(host='192.168.1.243',
                      user='root', port=3306, password='big2019',db='acornschool',
                       charset='utf8mb4', use_unicode=True,
                       cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()

data = ('귀염', 0)
cur.callproc('pet_delete', data)
cur.execute('SELECT @_pet_delete_1')
conn.commit()
result= cur.fetchone()
print(result['@_pet_delete_1'])
if result['@_pet_delete_1'] == 2:
    print('수정할 이름이 없습니다')
else:
    print('성공적으로 수정되었습니다')


cur.close()
conn.close()

2
수정할 이름이 없습니다


# try ~ except

In [51]:
conn = pymysql.connect(host='192.168.1.243',
                      user='root', port=3306, password='big2019',db='acornschool',
                       charset='utf8mb4', use_unicode=True,
                       cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()

try:   # exception 예외처리 (프로그램은 지속적으로 실행되어야 하기 때문에 예외처리를 헤준다. )
    data = ('귀염', 0)
    cur.callproc('pet_delete', data)
    cur.execute('SELECT @_pet_delete_1')
    conn.commit()
    result= cur.fetchone()
    print(result['@_pet_delete_1'])
    if result['@_pet_delete_1'] == 2:
        print('수정할 이름이 없습니다')
    else:
        print('성공적으로 수정되었습니다')
except :
    db.rollback()      # 예외 들이 except로 온다. ( 여기에 지정한 값으로 처리된다. )

cur.close()
conn.close()

2
수정할 이름이 없습니다
